# 課題 スクレイピングを業務で活用するアイデアを考えてみよう

[Yahoo! JAPANのトップページ](https://www.yahoo.co.jp/)から主要ニュースのリストを取得し、Markdown形式に変換して表示する。

## LLMを使わない場合

In [10]:
import requests
from bs4 import BeautifulSoup
from typing import TypedDict

class News(TypedDict):
  title: str
  url: str

def fetch_news_list() -> list[News]:
  response = requests.get("https://www.yahoo.co.jp/")
  response.raise_for_status()
  soup = BeautifulSoup(response.content, "html.parser")
  news_links = soup.select('#tabpanelTopics1 [aria-label="主要 ニュース"] a[href^="https://news.yahoo.co.jp/"]')
  news_list = []

  for link in news_links:
    title = link.select_one('h1').get_text(strip=True)
    url = link.get('href')
    news: News = {"title": title, "url": url}
    news_list.append(news)

  return news_list

def to_markdown(news_list: list[News]) -> str:
  markdown_lines = []

  for news in news_list:
    markdown_lines.append(f"- [{news['title']}]({news['url']})")

  return "\n".join(markdown_lines)

def fetch_news_list_not_using_llm() -> str:
  news_list = fetch_news_list()
  return to_markdown(news_list)

if __name__ == "__main__":
  markdown_not_using_llm = fetch_news_list_not_using_llm()
  print(markdown_not_using_llm)

- [備蓄米 定期販売も選択肢と小泉氏](https://news.yahoo.co.jp/pickup/6551885)
- [総裁選 平デジタル相が派閥けん制](https://news.yahoo.co.jp/pickup/6551883)
- [立駐でベビーカー転落 0歳児搬送](https://news.yahoo.co.jp/pickup/6551882)
- [ラーメン店強殺 起訴内容一部否認](https://news.yahoo.co.jp/pickup/6551888)
- [偽のミャクミャクT 所持疑い逮捕](https://news.yahoo.co.jp/pickup/6551879)
- [セブンがロボット試験導入 省人化](https://news.yahoo.co.jp/pickup/6551887)
- [101回目続編 抵抗感じる人いる訳](https://news.yahoo.co.jp/pickup/6551876)
- [吉田照美 体調不良で活動休止](https://news.yahoo.co.jp/pickup/6551878)


## LLMを使う場合

In [3]:
import requests
from bs4 import BeautifulSoup
import os
from dotenv import load_dotenv
from openai import OpenAI

MODEL_NAME = "gpt-4o-mini"

def fetch_news_list_using_llm() -> str:
  response = requests.get("https://www.yahoo.co.jp/")
  response.raise_for_status()
  soup = BeautifulSoup(response.content, "html.parser")
  body = str(soup.body)
  load_dotenv("../.env")
  client = OpenAI(api_key=os.environ['API_KEY'])
  prompt = f"""
  以下のHTMLから主要ニュースを抽出し、タイトルとURLを以下のようなMarkdown形式で出力してくだい。
  一覧以外（コードブロック用の```など）は出力しないでください。

  出力形式:
  - [タイトル](URL)

  HTML:
  ```html
  {body[:100000]}
  ```
  """
  response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
      {"role": "user", "content": prompt},
    ],
    max_tokens=500,
    temperature=0.3
  )
  return response.choices[0].message.content.strip()

if __name__ == "__main__":
  markdown_using_llm = fetch_news_list_using_llm()
  print(markdown_using_llm)

- [備蓄米 定期販売も選択肢と小泉氏](https://news.yahoo.co.jp/pickup/6551885)
- [総裁選 平デジタル相が派閥けん制](https://news.yahoo.co.jp/pickup/6551883)
- [ネパール政府 SNS禁止令を解除](https://news.yahoo.co.jp/pickup/6551863)
- [立駐でベビーカー転落 0歳児搬送](https://news.yahoo.co.jp/pickup/6551882)
- [偽のミャクミャクT 所持疑い逮捕](https://news.yahoo.co.jp/pickup/6551879)
- [セブンがロボット試験導入 省人化](https://news.yahoo.co.jp/pickup/6551887)
- [101回目続編 抵抗感じる人いる訳](https://news.yahoo.co.jp/pickup/6551876)
- [吉田照美 体調不良で活動休止](https://news.yahoo.co.jp/pickup/6551878)


## テスト

In [4]:
assert markdown_not_using_llm == markdown_using_llm